sql code to create efi2 deficit for cog impairement

In [1]:
#query for efii2 deficit for cognitive impiarement
 CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efi2_ci` as (
#bring all snomed and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  ctv3code,
  snomedcode,
  dateevent,
  cast(numericvalue as bignumeric) as risk_score
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    j.first_episodestartdate
    from a
    left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j on
    a.person_id = j.person_id),
#filter for cognitive impiarement snomed codeswithout numericvalues
  b as (
    select
    person_id,
    snomedcode,
    dateevent
    from a_master
    where snomedcode in(SELECT SNOMEDCT_CONCEPTID FROM `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Cognitive impairment' AND Otherinstructions IS NULL) AND dateevent < first_episodestartdate),
#filter for cog impairement with snomed codes 
  c as (
    select
    person_id,
    snomedcode, 
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Cognitive impairment' AND Otherinstructions IS NOT NULL) AND dateevent < first_episodestartdate AND risk_score >= 8),
  d as (
    select
    person_id, 
    ctv3code,
    dateevent
    from a_master
    where ctv3code in (select ctv3 FROM `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Cognitive impairment' AND Otherinstructions IS NULL)
  ),
#union distinct individuals identified from ctv3 and snomed ct codes together
  snomed_ctv3 as (
    select
    * from b

    union distinct

    select *
    from c
    
    union distinct
    
    select *
    from d)
#apply dummy variable
    select
    distinct person_id,
    case when dateevent IS NOT NULL then 'yes'
    end as cognitive_impairement
    from snomed_ctv3)
  





ERROR: Error in parse(text = x, srcfile = src): <text>:2:9: unexpected symbol
1: #query for efii2 deficit for cognitive impiarement
2:  CREATE OR
           ^
